In [3]:
from datasets import load_dataset
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim import models
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

dataset = load_dataset('mlsum', 'fr')

  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
article = dataset['test']['text'][1]
summary = dataset['test']['summary'][1]
title = dataset['test']['title'][1]
print(article, summary, title, sep='\n\n')

nlp = spacy.load("fr_core_news_sm")
doc = nlp(article)

sentences = []
cur_sentence = []
for sent in doc.sents:
    for token in sent:
        if not token.text.lower() in STOP_WORDS and not token.is_punct:
            cur_sentence.append(token.lemma_)
    sentences.append(cur_sentence)
    cur_sentence = []
    
dictionary = corpora.Dictionary(sentences)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in sentences]
tfidf = models.TfidfModel(doc_term_matrix)
sentences_tfidf = tfidf[doc_term_matrix]

coherence_values = []
model_list = []
for num_topics in range(2, 10):
    model = LsiModel(sentences_tfidf, num_topics=num_topics, id2word=dictionary)
    model_list.append(model)
    coherencemodel = CoherenceModel(model=model, texts=sentences, dictionary=dictionary)
    coherence_values.append(coherencemodel.get_coherence())

max_coherence = coherence_values.index(max(coherence_values))
num_topics = 2 + max_coherence
model = model_list[max_coherence]
corpus_lsi = model[doc_term_matrix]

top_scores = [[] for i in range(num_topics)]
for i, scores in enumerate(corpus_lsi):
    for j, score in scores:
        top_scores[j].append((i, abs(score)))

for topic in top_scores:
    topic.sort(reverse=True, key=lambda x: x[1])

print(num_topics)

Michel Serres, en mai 2012. MANUEL COHEN Parmi les figures multiples de Michel Serres, mort samedi 1er juin, à l’âge de 88 ans, il en est une qui réunit, peut-être mieux que d’autres, les aspects principaux de ce penseur. Il était français, et développait les conséquences intellectuelles et stylistiques de cette filiation. Ce n’est pas une question d’état civil, ni de chauvinisme culturel. Une affaire de style plutôt, et de tempérament. Pas simple à définir en deux mots, d’ailleurs, car les Français, selon lui, ont trouvé le moyen de faire de l’unité avec de la discorde. Dans ce pays de paradoxes, qui tend vers l’universel à force de divisions, les philosophes ne sont pas comme ailleurs. Michel Serres, bien que couvert d’honneurs, se rêvait comme l’un de ces marginaux nomades battant la campagne et franchissant les clôtures, braconnier et bretteur plutôt que professeur ou dignitaire. Français, donc frondeur. Et animé aussi d’un goût charnel de la langue natale, des puissances tapies da

In [33]:
def get_top_sentences(summary_size=5):
    top_sentences = set()
    count = 0
    for i in range(summary_size):
        for j in range(num_topics):
            if i >= len(top_scores[j]):
                continue
            top_sentences.add(top_scores[j][i][0])
            if len(top_sentences) == summary_size:
                return sorted(top_sentences)
    return sorted(top_sentences)

sents = list(doc.sents)
longest_summary = ''
for i in range(1, len(sents) + 1):
    top_sentences = get_top_sentences(i)
    summary = ""
    for sent_idx in top_sentences:
        summary += sents[sent_idx].text + " "
    if len(summary) > 280:
        break
    longest_summary = summary

print(longest_summary)

C’est au contraire comme un vagabond ami de la Terre, arpenteur inventif, ouvert aux fécondités du hasard. Michel Serres aura sans doute rendu à la philosophie française 
